In [48]:
import os
from urllib.request import Request, urlopen
from urllib.error import URLError
import requests, wget, re
from time import sleep
import configuration as config
from pathlib import Path

from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem

# you can also import SoftwareEngine, HardwareType, SoftwareType, Popularity from random_user_agent.params
# you can also set number of user agents required by providing `limit` as parameter

software_names = [SoftwareName.CHROME.value]
operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   

user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)

# Get list of user agents.
user_agents = user_agent_rotator.get_user_agents()

# Get Random User Agent String.
user_agent = user_agent_rotator.get_random_user_agent()
#print(user_agent)

In [49]:
webpage_dict = {}
url_list = []

download=False
i=0 
#get search urls / kbb only? 
with open(join(config.INPUT_DIR, 'url_list.txt'), 'r') as f:
    search_url_list = f.readlines()

#find urls in a kbb search page
for search_url in search_url_list:
    txt = requests.get(search_url).text
    found_urls = re.findall(r'"url":(".*?")', txt)

    for item in found_urls:
        if 'listingId' not in item:
            found_urls.remove(item)
        if len(item) < 10:
            found_urls.remove(item)
    url_list.extend(found_urls[:-4])


for url in url_list:
    print(i, end=" ")
    i+=1
    try:
        url = url.split('"')[1]
    except:
        pass
    req = Request(url, headers={'User-Agent':user_agent_rotator.get_random_user_agent(),
                                'accept':'text/html;q=0.8,application/signed-exchange;v=b3;q=0.9',
                                'accept-language':'en-US,en;q=0.9'})
    try:
        response = urlopen(req)
    except URLError as e:
        print(url)
        if hasattr(e, 'reason'):
            print('We failed to reach a server.')
            print('Reason: ', e.reason)
        elif hasattr(e, 'code'):
            print('The server couldn\'t fulfill the request')
            print('Error code:', e.code)
    else:
        try:
            if response.headers.get_content_charset() == 'utf-8':
                webpage_as_string = response.read().decode(response.headers.get_content_charset(), "ignore")
            else:
                webpage_as_string = response.read().decode(response.headers.get_content_charset(), "ignore")
        except UnicodeEncodeError as e:
            print(e)
            webpage_as_string = response.read()
        #print(webpage_as_string)
        #print(response.headers.get_content_charset())
        
        if download == True:
            filename = url.split('//')[1].split('/')[0]
            with open(filename+'.txt', 'wb') as f:
                f.write(webpage_as_string.encode('ascii', 'ignore'))
                
        webpage_dict[url] = (webpage_as_string)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 https://www.kbb.com/cars-for-sale/vehicledetails.xhtml?listingId=592771169
We failed to reach a server.
Reason:  Forbidden
46 47 48 49 50 51 52 53 

In [52]:
import requests
import json
from ast import literal_eval
import pandas as pd
from os.path import join

output_filename = join(config.OUTPUT_DIR, f"potential vehicles.csv")
vehicle_df = pd.DataFrame(columns=["Price", "Mileage", "Year", "Make", "Model", "Trim", 
                            "Color", "Fuel Economy", "VIN", "Location", "URL"])

if os.path.exists(output_filename):
    vehicle_df = pd.read_csv(output_filename)

for url, txt in webpage_dict.items():
    #print(url)
    with open("debug.txt", "w") as f: f.write(txt)
    base_url = '/'.join(url.split('/')[0:3])

        #kbb data scraper
    if 'kbb' in base_url:  
            #find vehicle data
        vehicle_dict = {}
        url = url.split("&")[0]
        try:
            location_dict = literal_eval(re.findall(r'"location":({.*?})', txt)[1]+"}")['address']
            vehicle_data_dict = literal_eval(re.findall(r'"vehicle":({.*?})', txt)[0])
        except:
            continue

            #put vehicle data into dict
        #address = location_dict['address1']
        city = location_dict['city']
        state = location_dict['state']
        zip_code = location_dict['zip']

        full_address = f"{city}, {state} {zip_code}"
            
        vehicle_dict["Price"] = vehicle_data_dict['price']
        vehicle_dict["Mileage"] = int("".join(vehicle_data_dict['odometer'].split(',')).split(" ")[0])
        vehicle_dict["Year"] = vehicle_data_dict['car_year']
        vehicle_dict["Make"] = vehicle_data_dict['makeName'][0]
        vehicle_dict["Model"] = vehicle_data_dict['modelName'][0]
        vehicle_dict["Color"] = vehicle_data_dict['color'][0]
        vehicle_dict["Fuel Economy"] = " | ".join(vehicle_data_dict['fuelEconomy'])
        vehicle_dict["VIN"] = vehicle_data_dict['vin']
        vehicle_dict["Location"] = full_address
        vehicle_dict["URL"] = url
        
        try:
            vehicle_dict["Trim"] = vehicle_data_dict['trim']
        except:
            pass

        vehicle_df = vehicle_df.append(vehicle_dict, ignore_index=True)

    #add vehicle_dict to dataframe and save in csv
    elif 'dat boi' in base_url:
        pass

vehicle_df = vehicle_df.drop_duplicates()
print(len(vehicle_df))
vehicle_df.to_csv(join(output_filename), index=False)


61
